In [1]:
%pip install nltk

You should consider upgrading via the '/Users/texnedo/Documents/projects/mathematics-and-python/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import nltk
from nltk import bigrams, trigrams
from nltk.corpus import reuters
from collections import defaultdict
from nltk.probability import FreqDist
import random
import numpy as np
import re
import requests

# Download necessary NLTK resources
nltk.download('reuters')
nltk.download('punkt_tab')

/Users/texnedo/Documents/projects/mathematics-and-python/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package reuters to /Users/texnedo/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/texnedo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
#reuters.words()

In [4]:
response = requests.get("https://raw.githubusercontent.com/mmcky/nyu-econ-370/refs/heads/master/notebooks/data/book-war-and-peace.txt")
print("HTTP response: " + str(response.status_code))

HTTP response: 200


In [5]:
def create_model(words):
    print("Number of words: " + str(len(words)))

    # Create trigrams
    tri_grams = list(trigrams(words))

    # Build a trigram model
    model = defaultdict(lambda: defaultdict(lambda: 0))

    # Count frequency of co-occurrence
    for w1, w2, w3 in tri_grams:
        model[(w1, w2)][w3] += 1

    # Transform the counts into probabilities
    for w1_w2 in model:
        total_count = float(sum(model[w1_w2].values()))
        for w3 in model[w1_w2]:
            model[w1_w2][w3] /= total_count
    return model

In [6]:
data = response.text
data[0:100]

'CHAPTER I\n\n"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I w'

In [7]:
# Tokenize the text
#words = nltk.word_tokenize(' '.join(reuters.words()))
words_processed = nltk.word_tokenize(response.text.lower())
words = []
for word in words_processed:
    words.append(word)
    if word in ['.', '!', '?']:
        words.append('<s>')
model = create_model(words)

Number of words: 700250


In [8]:
words[0:100]

['chapter',
 'i',
 "''",
 'well',
 ',',
 'prince',
 ',',
 'so',
 'genoa',
 'and',
 'lucca',
 'are',
 'now',
 'just',
 'family',
 'estates',
 'of',
 'the',
 'buonapartes',
 '.',
 '<s>',
 'but',
 'i',
 'warn',
 'you',
 ',',
 'if',
 'you',
 'do',
 "n't",
 'tell',
 'me',
 'that',
 'this',
 'means',
 'war',
 ',',
 'if',
 'you',
 'still',
 'try',
 'to',
 'defend',
 'the',
 'infamies',
 'and',
 'horrors',
 'perpetrated',
 'by',
 'that',
 'antichrist',
 '--',
 'i',
 'really',
 'believe',
 'he',
 'is',
 'antichrist',
 '--',
 'i',
 'will',
 'have',
 'nothing',
 'more',
 'to',
 'do',
 'with',
 'you',
 'and',
 'you',
 'are',
 'no',
 'longer',
 'my',
 'friend',
 ',',
 'no',
 'longer',
 'my',
 "'faithful",
 'slave',
 ',',
 "'",
 'as',
 'you',
 'call',
 'yourself',
 '!',
 '<s>',
 'but',
 'how',
 'do',
 'you',
 'do',
 '?',
 '<s>',
 'i',
 'see',
 'i',
 'have']

In [9]:
# Compute the frequency distribution
fdist = FreqDist(words)

# Total number of words in the corpus
total_words = sum(fdist.values())

print("Number of words: " + str(total_words))

Number of words: 700250


In [10]:
# Function to predict the next word
def predict_next_word(w1, w2):
    """
    Predicts the next word based on the previous two words using the trained trigram model.
    Args:
    w1 (str): The first word.
    w2 (str): The second word.

    Returns:
    Tuple = (str: The predicted next word, float: Probability, object: All possible combinations)
    """
    next_word = model[w1, w2]
    if next_word:
        predicted_word = max(next_word, key=next_word.get)  # Choose the most likely next word
        return (predicted_word, next_word[predicted_word], next_word)
    else:
        return None

In [11]:
def get_unigram_freq(word):
    if word not in fdist:
        return None
    count = fdist[word]
    return count / total_words

In [12]:
def calculate_perplexity(probabilities):
    """
    Calculate perplexity given an array of word probabilities in a sentence.
    
    Args:
        probabilities (list or numpy.ndarray): Array of probabilities assigned by the model 
                                                to each word in the sentence. 
                                                Each probability should be a value between 0 and 1.
                                                
    Returns:
        float: Perplexity score for the sentence.
    """
    if not probabilities or any(p <= 0 or p > 1 for p in probabilities):
        raise ValueError("Probabilities must be a non-empty array of values between 0 and 1.")
    
    log_probs = np.log(probabilities)
    avg_log_prob = np.mean(log_probs)
    perplexity = np.exp(-avg_log_prob)
    
    return perplexity

In [13]:
def is_token_without_punctuation(token):
    """
    Check if a token contains no punctuation marks.
    
    Args:
        token (str): The token to check.
    
    Returns:
        bool: True if the token has no punctuation, False otherwise.
    """
    pattern = r"^[^\W_]*$"
    return bool(re.match(pattern, token, flags=re.UNICODE))

In [14]:
generated_count = 0
used_trigrams = set()
while generated_count < 50:
    #i_word = words[random.randint(0, len(words) - 1)]
    i_word = '<s>'
    i_1_word = words[random.randint(0, len(words) - 1)]
    #print("Stating from: [" + i_word + " " + i_1_word + "]")
    #if not is_token_without_punctuation(i_word) or not is_token_without_punctuation(i_1_word):
    #    continue
    if not is_token_without_punctuation(i_1_word):
        continue
    result = [i_word, i_1_word]
    result_probs = [get_unigram_freq(i_word), get_unigram_freq(i_1_word)]
    success = True
    for i in range(2, 30):
        text = predict_next_word(result[i - 2], result[i - 1])
        if text is None:
            success = False
            break
        current_trigram = (result[i - 2], result[i - 1], text[0])
        if current_trigram in used_trigrams:
            success = False
            break 
        used_trigrams.add(current_trigram)
        result.append(text[0])
        if text[0] in {'.', '!', '?', ';'}:
            break
    if success:
        score = calculate_perplexity(result_probs)
        if score > 1000:
            continue
        sentence = ' '.join(result).replace('<s>', '')
        print("Generared sentence: " + sentence)
        print("Perplexity score: {}".format(score))
        generated_count += 1
        

Generared sentence:  an adjutant to the emperor 's eyes .
Perplexity score: 97.31231502375717
Generared sentence:  with a smile .
Perplexity score: 52.19213536274845
Generared sentence:  too soon for news .
Perplexity score: 198.3742569489315
Generared sentence:  enough !
Perplexity score: 502.8789177580497
Generared sentence:  fine !
Perplexity score: 355.58909286246904
Generared sentence:  ca n't be helped !
Perplexity score: 280.54356460286414
Generared sentence:  since the ball .
Perplexity score: 286.4503926591006
Generared sentence:  followed by a new and unexpected steps .
Perplexity score: 338.0348388954622
Generared sentence:  how can you judge of it .
Perplexity score: 124.07791464901662
Generared sentence:  what is it ?
Perplexity score: 80.33955022996527
Generared sentence:  hardly had he not been for the first time .
Perplexity score: 416.3258172198484
Generared sentence:  listen !
Perplexity score: 416.3258172198484
Generared sentence:  can i do n't know what to do so .
P

In [15]:
input_text = "war"
result = predict_next_word("<s>", input_text)

In [16]:
print(result)

('is', 1.0, defaultdict(<function create_model.<locals>.<lambda>.<locals>.<lambda> at 0x16a68e310>, {'is': 1.0}))


In [17]:
from ngram_processing_utils import *

In [48]:
tokenized_data = get_tokenized_data(response.text, skip_words = {"chapter", ",", ".", "'s"})

In [49]:
len(response.text)

3202303

In [50]:
len(tokenized_data)

50886

In [51]:
for item in tokenized_data:
    for word in item:
        if "nataly" in word:
            print(item)

['mother', 'and', 'the', 'youngest', 'daughter', '--', 'both', 'named', 'nataly', 'ever', 'since', 'the']
['``', 'natalya', 'ilynichna', 'behaves', 'very', 'well', 'to', 'me', "''", 'remarked', 'boris', '``', 'i', 'have']
['nataly', 'you', 'know', 'my', 'love', 'for', 'my', 'son', ':', 'i', 'would', 'do', 'anything', 'for', 'his']
['``', 'that', 'is', 'with', 'ilya', 'rostov', 'who', 'married', 'nataly', 'shinshina', "''", 'said', 'anna']
['never', 'could', 'understand', 'how', 'nataly', 'made', 'up', 'her', 'mind', 'to', 'marry', 'that']
['``', 'nataly', "''", 'he', 'said', 'moving', 'with', 'rapid', 'steps', 'toward', 'her', '``', 'decide', 'my', 'fate']
['``', 'natalya', 'ilynichna', "''", 'pierre', 'began', 'dropping', 'his', 'eyes', 'with', 'a', 'feeling', 'of']


In [52]:
for item in tokenized_data:
    for word in item:
        if "rostova" in word:
            print(item)

['continually', 'bringing', 'visitors', 'to', 'the', 'countess', 'rostova', 'big', 'house', 'on']
['rostova', 'carriage', 'in', 'which', 'they', 'were', 'seated', 'drove', 'over', 'the', 'straw']
['``', 'i', 'am', 'living', 'at', 'countess', 'rostova', "''", 'replied', 'boris', 'again', 'adding', '``', 'your']
['vladimirovich', 'bezukhov', 'countess', 'rostova', 'sat', 'for', 'a', 'long', 'time', 'all', 'alone']
['countess', 'rostova', 'with', 'her', 'daughters', 'and', 'a', 'large', 'number', 'of', 'guests', 'was']
['theater', 'berg', 'had', 'pointed', 'out', 'vera', 'rostova', 'to', 'him', 'and', 'had', 'said', 'in']
['livonian', 'gentleman', 'should', 'propose', 'marriage', 'to', 'a', 'countess', 'rostova', ';', 'but']
['``', 'you', 'always', 'dance', 'i', 'have', 'a', 'protegee', 'the', 'young', 'rostova', 'here', 'ask', 'her', "''"]
['pleasure', 'on', 'his', 'face', 'approached', 'countess', 'rostova']
['little', 'rostova', 'is', 'very', 'charming', 'there', 'something', 'fresh', 

In [53]:
minimum_freq = 1
train_data_processed, test_data_processed, vocabulary, word_counts = preprocess_data(tokenized_data, 
                                                                        tokenized_data, 
                                                                        minimum_freq)

In [54]:
all_words = []
all_words_counts = []
for i, j in word_counts.items():
    all_words.append((i, j))
    all_words_counts.append(j)
indexes = np.argsort(all_words_counts)
for i in range(1, 100):
    print(all_words[indexes[-i]])

('the', 34533)
('and', 22208)
('to', 16667)
('of', 14885)
('a', 10540)
('he', 9999)
("''", 9010)
('``', 8939)
('in', 8779)
('that', 8186)
('his', 7984)
('was', 7372)
('with', 5663)
('it', 5596)
('had', 5373)
('not', 4840)
('her', 4725)
('him', 4637)
('at', 4531)
('i', 4522)
('but', 4045)
('as', 4023)
('on', 4000)
('!', 3923)
('you', 3793)
('for', 3517)
('she', 3488)
('is', 3365)
('?', 3136)
('said', 2842)
('all', 2785)
('from', 2687)
('be', 2438)
('were', 2431)
('what', 2390)
('by', 2383)
('they', 2251)
('who', 2162)
('one', 2119)
('--', 2083)
('this', 2073)
('which', 2057)
('have', 2002)
('pierre', 1963)
('prince', 1927)
('so', 1861)
('an', 1629)
('do', 1564)
('there', 1557)
('up', 1553)
('did', 1535)
('them', 1528)
('or', 1523)
('when', 1494)
('been', 1476)
('...', 1447)
('their', 1440)
('no', 1396)
('would', 1384)
('now', 1332)
('only', 1298)
('if', 1292)
('are', 1286)
('me', 1273)
('out', 1233)
('my', 1225)
('natasha', 1213)
('man', 1189)
("n't", 1155)
(';', 1145)
('andrew', 1143)


In [55]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [56]:
previous_tokens = ["natasha", "rostova", "and", "pierre", "bezukhov", "well", ',','and', 'what']
tmp_suggest = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="")
print(tmp_suggest)

[('he', 0.010702405630815214), ('is', 0.0007039202945635694), ('i', 5.448700484934343e-05), ('i', 5.448700484934343e-05)]


In [57]:
previous_tokens = ["kutuzov", "'s", "staff", "officer"]
tmp_suggest = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="")
print(tmp_suggest)

[('<e>', 0.0025009311977864097), ('and', 0.0003264062670003264), ('i', 5.448700484934343e-05), ('i', 5.448700484934343e-05)]


In [67]:
previous_tokens = ["pierre", "was", "so", "deep", "in", "thought", "of", "the", "battle"]
tmp_suggest = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="")
print(tmp_suggest)

[('of', 0.003955738493612017), ('of', 0.00340374952725701), ('of', 0.0007612833061446438), ('i', 5.448700484934343e-05)]


In [74]:
previous_tokens = ["natasha", "was", "in", "a", "state", "of", "affairs"]
tmp_suggest = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="")
print(tmp_suggest)

[('and', 0.0009202620040058464), ('and', 0.000326477309826967), ('and', 0.0001633808953273064), ('i', 5.448700484934343e-05)]


In [81]:
previous_tokens = ["natasha", "and", "pierre", "were", "living", "in", "petersburg"]
tmp_suggest = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="")
print(tmp_suggest)

[('and', 0.0014519251451925144), ('and', 0.0004883872368135446), ('at', 0.00010896807235480004), ('at', 0.00010896807235480004)]
